In [133]:
import pandas as pd

In [134]:
folder = './input_files/'

file = '46-0319070_RapidCityHospital_StandardCharges.xlsx'
# file = '46-0360899_SpearfishHospital_StandardCharges.xlsx'

In [135]:
df = pd.read_excel(folder + file, dtype=str, sheet_name='Bundled Services', skiprows=1)

In [136]:
df.drop(columns=['Location', 'Charge'], inplace=True)

In [137]:
df.rename(columns={
    'Code Type': 'line_type',
    'Code': 'code',
    'Description': 'description'
}, inplace=True)

In [139]:
df.loc[~df['Blue Cross Blue Shield Wellmark\nBlue Cross Blue Shield Federal'].isna()]

,line_type,code,description,Patient Type,Blue Cross Blue Shield Wellmark\nBlue Cross Blue Shield Federal,HealthPartners- Monument,DakotaCare,Blue Cross Medicare Blue,Good Samaritin,Humana Choice (Medicare)\nHumana Gold Plus (Medicare),United Health Care Medicare,De-Identified Minimum Negotiated Charge,De-Identified Maximum Negotiated Charge
671,APR-DRG,004,TRACHEOSTOMY WITH MV >96 HOURS WITH EXTENSIVE ...,Inpatient,158899.73,NaN,NaN,NaN,NaN,NaN,NaN,158899.73,158899.73
672,APR-DRG,005,TRACHEOSTOMY WITH MV >96 HOURS WITHOUT EXTENSI...,Inpatient,124324.41,NaN,NaN,NaN,NaN,NaN,NaN,124324.41,124324.41
673,APR-DRG,009,EXTRACORPOREAL MEMBRANE OXYGENATION (ECMO),Inpatient,123421.59,NaN,NaN,NaN,NaN,NaN,NaN,123421.59,123421.59
674,APR-DRG,020,OPEN CRANIOTOMY FOR TRAUMA,Inpatient,45221.89,NaN,NaN,NaN,NaN,NaN,NaN,45221.89,45221.89
675,APR-DRG,021,OPEN CRANIOTOMY EXCEPT TRAUMA,Inpatient,47383.98,NaN,NaN,NaN,NaN,NaN,NaN,47383.98,47383.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1502,EAPG,3035,Abdominal Hernia Repair,Outpatient,9532.6,NaN,NaN,NaN,NaN,NaN,NaN,9532.6,9532.6
1503,EAPG,3050,Transplant Procedures,Outpatient,12492.48,NaN,NaN,NaN,NaN,NaN,NaN,12492.48,12492.48
1504,EAPG,4001,Emerging Technology Procedures,Outpatient,2086.7,NaN,NaN,NaN,NaN,NaN,NaN,2086.7,2086.7
1505,EAPG,4010,Level I Opioid Treatment Program Services,Outpatient,1200.43,NaN,NaN,NaN,NaN,NaN,NaN,1200.43,1200.43


In [140]:
def split_column_by_newline(df):
    for column_name in df.columns:
        if '\n' in column_name:
            new_column_names = column_name.split('\n')
            df[new_column_names[0]] = df[column_name]
            df[new_column_names[1]] = df[column_name]
            df.drop(column_name, axis=1, inplace=True)
            
# Apply the function to split columns with '\n'
split_column_by_newline(df)

In [141]:
cols = df.columns.tolist()
id_vars = cols[:4]
val_vars = cols[4:]

df = pd.melt(df, id_vars=id_vars, value_vars=val_vars, var_name='payer_name', value_name='standard_charge')

In [126]:
payer_mapping = {
    'De-Identified Minimum Negotiated Charge': 'min',
    'De-Identified Maximum Negotiated Charge': 'max',
}

df['rate_category'] = df['payer_name'].map(payer_mapping).fillna('negotiated')

In [127]:
df.loc[df['line_type'].str.upper().str.contains('PER DIEM'), 'contracting_method'] = 'per diem'

In [128]:
df.loc[df['line_type'] == 'MS-DRG', 'ms_drg'] = df['code']
df.loc[df['line_type'] == 'MS DRG-PER DIEM', 'ms_drg'] = df['code']
df.loc[df['line_type'] == 'APR-DRG', 'apr_drg'] = df['code']
df.loc[df['line_type'] == '	APR-DRG Per Diem', 'apr_drg'] = df['code']
df.loc[df['line_type'] == 'EAPG', 'eapg'] = df['code']
df.loc[df['line_type'] == 'APC', 'apc'] = df['code']


In [129]:
df.loc[df['Patient Type'].str.contains('Inpatient'), 'setting'] = 'inpatient'
df.loc[df['Patient Type'].str.contains('Outpatient'), 'setting'] = 'outpatient'

In [130]:
id_mapping = {
'46-0360899_SpearfishHospital_StandardCharges.xlsx': '430048',
'46-0319070_RapidCityHospital_StandardCharges.xlsx': '430077',
'46-0360899_LeadDeadwoodHospital_StandardCharges.xlsx': '431320',
'46-0360899_SturgisHospital_StandardCharges.xlsx': '431321',
'46-0360899_CusterHospital_StandardCharges.xlsx': '431323'
}

hosp_id = id_mapping[file]

df['hospital_id'] = hosp_id

output_folder = '.\\output_files\\'

df.to_csv(output_folder + hosp_id + '_' + file.split('_')[1] + '_' + 'bundled_services' + '.csv', index=False)
